Now we can try to fine tune the data

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 1725/1725 [00:00<00:00, 20482.55 examples/s]


For training, we just need to pass the models where the trained model will be saved

In [4]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)



/var/folders/qv/63mfhrln5q909zb9cygj6ry40000gn/T/ipykernel_88091/2700434406.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


We can train the data 

In [7]:
trainer.train()

Step,Training Loss
500,0.537200
1000,0.371500


TrainOutput(global_step=1377, training_loss=0.4072956592169203, metrics={'train_runtime': 518.5037, 'train_samples_per_second': 21.223, 'train_steps_per_second': 2.656, 'total_flos': 405114969714960.0, 'train_loss': 0.4072956592169203, 'epoch': 3.0})